In [1]:
import bs4
import pandas as pd
import requests
import re

In [2]:
url="https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250"

In [3]:
req=requests.get(url)

In [4]:
req

<Response [200]>

In [5]:
soup=bs4.BeautifulSoup(req.text,'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <style>
   body#styleguide-v2 {
                    background: no-repeat fixed center top #000;
                }
  </style>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   IMDb Top 250 TV - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
  </script>
  <link href="http

# Find the title of the movie:

find(name,attrs={})

Look in the children of this PageElement and find the first PageElement that matches the given criteria

In [6]:
#st=show title
#attrs=attributes
st=soup.find('td',attrs={'class':'titleColumn'})
print(st)

<td class="titleColumn">
      1.
      <a href="/title/tt5491994/" title="David Attenborough, Gordon Buchanan">Planet Earth II</a>
<span class="secondaryInfo">(2016)</span>
</td>


# Scrap all Title,Rank,Crew,Year:

In [7]:
titles=[]
ranks=[]
crew=[]
year=[]
for i in soup.findAll('td',attrs={'class':'titleColumn'}):
    titles.append(i.a.text)                      #show name
    ranks.append(i.text.strip()[:3])             #rank
    crew.append(i.a.get('title'))                #cast and crew  
    year.append(i.span.text)                     #release_year

In [8]:
print(len(titles))
print(len(ranks))
print(len(crew))
print(len(year))

250
250
250
250


In [9]:
ratings=[]
user_rated=[]
for i in soup.findAll('td',attrs={'class':'ratingColumn imdbRating'}):
    ratings.append(i.strong.text)                                #average rating
    user_rated.append(i.strong.get('title')[13:22])             #no of users voted

In [10]:
print(len(ratings))
print(len(user_rated))

250
250


# Create DataFrame:

In [11]:
import pandas as pd

In [12]:
imdb_top_250=pd.DataFrame({'Rank':ranks,'Title':titles,'Year':year,'Crew':crew,'Ratings':ratings,'Users_Voted':user_rated})
imdb_top_250

,Rank,Title,Year,Crew,Ratings,Users_Voted
0,1.\n,Planet Earth II,(2016),"David Attenborough, Gordon Buchanan",9.4,"149,332 u"
1,2.\n,Breaking Bad,(2008),"Bryan Cranston, Aaron Paul",9.4,"1,959,635"
2,3.\n,Planet Earth,(2006),"Sigourney Weaver, David Attenborough",9.4,"212,952 u"
3,4.\n,Band of Brothers,(2001),"Scott Grimes, Damian Lewis",9.4,"480,921 u"
4,5.\n,Chernobyl,(2019),"Jessie Buckley, Jared Harris",9.3,"790,042 u"
...,...,...,...,...,...,...
245,246,Foyle's War,(2002),"Michael Kitchen, Honeysuckle Weeks",8.4,"16,536 us"
246,247,Gintama,(2005),"Tomokazu Sugita, Daisuke Sakaguchi",8.4,"12,585 us"
247,248,Black Books,(2000),"Dylan Moran, Bill Bailey",8.4,"60,147 us"
248,249,The Great British Bake Off,(2010),"Paul Hollywood, Mary Berry",8.4,"11,424 us"


In [13]:
imdb_top_250.shape

(250, 6)

In [14]:
imdb_top_250.head(10)

,Rank,Title,Year,Crew,Ratings,Users_Voted
0,1.\n,Planet Earth II,(2016),"David Attenborough, Gordon Buchanan",9.4,"149,332 u"
1,2.\n,Breaking Bad,(2008),"Bryan Cranston, Aaron Paul",9.4,"1,959,635"
2,3.\n,Planet Earth,(2006),"Sigourney Weaver, David Attenborough",9.4,"212,952 u"
3,4.\n,Band of Brothers,(2001),"Scott Grimes, Damian Lewis",9.4,"480,921 u"
4,5.\n,Chernobyl,(2019),"Jessie Buckley, Jared Harris",9.3,"790,042 u"
5,6.\n,The Wire,(2002),"Dominic West, Lance Reddick",9.3,"350,210 u"
6,7.\n,Avatar: The Last Airbender,(2005),"Dee Bradley Baker, Zach Tyler Eisen",9.2,"328,230 u"
7,8.\n,Blue Planet II,(2017),"David Attenborough, Peter Drost",9.2,"42,832 us"
8,9.\n,The Sopranos,(1999),"James Gandolfini, Lorraine Bracco",9.2,"418,766 u"
9,10.,Cosmos: A Spacetime Odyssey,(2014),"Neil deGrasse Tyson, Christopher Emerson",9.2,"123,956 u"


# Replace Unnecessary Data:

In [15]:
imdb_top_250['Rank']=imdb_top_250['Rank'].str.replace('.\n','',regex=True)    #regex is regular expresstion=by default True
imdb_top_250.head()

,Rank,Title,Year,Crew,Ratings,Users_Voted
0,1,Planet Earth II,(2016),"David Attenborough, Gordon Buchanan",9.4,"149,332 u"
1,2,Breaking Bad,(2008),"Bryan Cranston, Aaron Paul",9.4,"1,959,635"
2,3,Planet Earth,(2006),"Sigourney Weaver, David Attenborough",9.4,"212,952 u"
3,4,Band of Brothers,(2001),"Scott Grimes, Damian Lewis",9.4,"480,921 u"
4,5,Chernobyl,(2019),"Jessie Buckley, Jared Harris",9.3,"790,042 u"


In [16]:
imdb_top_250['Users_Voted']=imdb_top_250['Users_Voted'].str.replace(',','')
imdb_top_250.head()

,Rank,Title,Year,Crew,Ratings,Users_Voted
0,1,Planet Earth II,(2016),"David Attenborough, Gordon Buchanan",9.4,149332 u
1,2,Breaking Bad,(2008),"Bryan Cranston, Aaron Paul",9.4,1959635
2,3,Planet Earth,(2006),"Sigourney Weaver, David Attenborough",9.4,212952 u
3,4,Band of Brothers,(2001),"Scott Grimes, Damian Lewis",9.4,480921 u
4,5,Chernobyl,(2019),"Jessie Buckley, Jared Harris",9.3,790042 u


In [17]:
imdb_top_250['Users_Voted']=imdb_top_250['Users_Voted'].str.replace('us','').str.replace('u','')
imdb_top_250.head()

,Rank,Title,Year,Crew,Ratings,Users_Voted
0,1,Planet Earth II,(2016),"David Attenborough, Gordon Buchanan",9.4,149332
1,2,Breaking Bad,(2008),"Bryan Cranston, Aaron Paul",9.4,1959635
2,3,Planet Earth,(2006),"Sigourney Weaver, David Attenborough",9.4,212952
3,4,Band of Brothers,(2001),"Scott Grimes, Damian Lewis",9.4,480921
4,5,Chernobyl,(2019),"Jessie Buckley, Jared Harris",9.3,790042


In [18]:
imdb_top_250.tail()

,Rank,Title,Year,Crew,Ratings,Users_Voted
245,246,Foyle's War,(2002),"Michael Kitchen, Honeysuckle Weeks",8.4,16536
246,247,Gintama,(2005),"Tomokazu Sugita, Daisuke Sakaguchi",8.4,12585
247,248,Black Books,(2000),"Dylan Moran, Bill Bailey",8.4,60147
248,249,The Great British Bake Off,(2010),"Paul Hollywood, Mary Berry",8.4,11424
249,250,Saikojiman Gwaenchanha,(2020),"Kim Soo-hyun, Seo Ye-Ji",8.4,27181


In [19]:
imdb_top_250['Year']=imdb_top_250['Year'].str.replace("(","").str.replace(")",'')
imdb_top_250.head()

C:\Users\HP Laptop\AppData\Local\Temp\ipykernel_760\1711509581.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  imdb_top_250['Year']=imdb_top_250['Year'].str.replace("(","").str.replace(")",'')


,Rank,Title,Year,Crew,Ratings,Users_Voted
0,1,Planet Earth II,2016,"David Attenborough, Gordon Buchanan",9.4,149332
1,2,Breaking Bad,2008,"Bryan Cranston, Aaron Paul",9.4,1959635
2,3,Planet Earth,2006,"Sigourney Weaver, David Attenborough",9.4,212952
3,4,Band of Brothers,2001,"Scott Grimes, Damian Lewis",9.4,480921
4,5,Chernobyl,2019,"Jessie Buckley, Jared Harris",9.3,790042


# Convert into Excel File:

In [20]:
imdb_top_250.to_excel('imdb_shows.xlsx',index=False)

# Read Excel File:

In [21]:
df=pd.read_excel('imdb_shows.xlsx')

In [22]:
df

,Rank,Title,Year,Crew,Ratings,Users_Voted
0,1.0,Planet Earth II,2016,"David Attenborough, Gordon Buchanan",9.4,149332
1,2.0,Breaking Bad,2008,"Bryan Cranston, Aaron Paul",9.4,1959635
2,3.0,Planet Earth,2006,"Sigourney Weaver, David Attenborough",9.4,212952
3,4.0,Band of Brothers,2001,"Scott Grimes, Damian Lewis",9.4,480921
4,5.0,Chernobyl,2019,"Jessie Buckley, Jared Harris",9.3,790042
...,...,...,...,...,...,...
245,246.0,Foyle's War,2002,"Michael Kitchen, Honeysuckle Weeks",8.4,16536
246,247.0,Gintama,2005,"Tomokazu Sugita, Daisuke Sakaguchi",8.4,12585
247,248.0,Black Books,2000,"Dylan Moran, Bill Bailey",8.4,60147
248,249.0,The Great British Bake Off,2010,"Paul Hollywood, Mary Berry",8.4,11424
